In [ ]:
import numpy as np
import scipy.stats as stats
import subprocess

In [ ]:
# Initialize true parameters
dim = 3
betas = [np.array(dim*[-3]), np.array(dim*[+0]), np.array(dim*[+3])]
sigma2 = 1

In [ ]:
# Generate data
rng = 20201124
np.random.seed(rng)
n = 100
xx = np.random.normal(loc=0.0, scale=1.0, size=(n, dim))
cc = np.random.randint(low=2, size=n)
yy = np.zeros(n)
for i in range(n):
    mu = np.dot(xx[i, :], betas[cc[i]])
    y = np.random.normal(loc=mu, scale=sigma2)
    yy[i] = y
# Save data
np.savetxt("../resources/csv/in/covs_lru.csv", xx, fmt='%1.5f')
np.savetxt("../resources/csv/in/data_lru.csv", yy, fmt='%1.5f')

In [ ]:
# Generate grid points
rng = 20201125
np.random.seed(rng)
yy_grid = np.arange(-5.0, +5.0, 0.1)
xx_grid = np.random.normal(loc=0.0, scale=1.0, size=(yy_grid.size, dim))
# Save grid points
np.savetxt("../resources/csv/in/covs_grid_lru.csv", xx_grid, fmt='%1.5f')
np.savetxt("../resources/csv/in/grid_lru.csv", yy_grid, fmt='%1.5f')

TODO:
* Compare the posterior similarity matrix with the true similarity matrix,
  the posterior number of clusters, the values of the betas at some iteration
  (they should match b_1, ... b_3 roughly).
* Check that the MSE on the training set is lower than the MSE you
  would get using a standard linear regression.
* True variance vs state.var

In [ ]:
# Generate grid
# grid_uni = np.arange(-10, +10, 0.1)
# np.savetxt("../resources/csv/in/grid_uni.csv", grid_uni, fmt='%1.5f')

In [ ]:
# True density of data
# true_pdf = 0.5 * stats.norm.pdf(grid_uni, -4.0, 1.0) + \
#            0.5 * stats.norm.pdf(grid_uni, +4.0, 1.0)

In [ ]:
# Run the executable
# cmd = ["../build/run",
#     "N8", str(rng), "0", "1000", "100",
#     "NNW", "../resources/asciipb/nnw_ngiw_prior.asciipb",
#     "DP", "../resources/asciipb/dp_gamma_prior.asciipb",
#     "../resources/csv/in/data_multi.csv", "../resources/csv/in/grid_multi.csv",
#     "../resources/csv/out/mn_dens.csv", "../resources/csv/out/mn_mass.csv",
#     "../resources/csv/out/mn_nclu.csv", "../resources/csv/out/mn_clus.csv"
# ]
# subprocess.run(cmd, capture_output=True)